점점 미쳐가는 중

###사유

7일의 데이터를 사용해서 2일의 데이터를 예측해야한다ㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏ


###문제

많은 것이 문제, 무엇보다 문제는 **input과 output의 모양이 달라서** 도통 되먹지를 않는다.

일단 7일치를 2일치로 퉁치는 것이 문제임

두 번째 문제는 변수 6개로 1개 예측하는 것이 문제임

합치면, **7일치의 변수 6개로, 2일치 정답변수 1개를 예측하는 문제**가 되어서 난장판ㄴㄴㄴㄴㄴ

문제2, 원래 LSTM은 이미지 처리, 텍스트 처리에 쓰는 경우가 많아서 일단 이런 회귀는 다른 모델이 더 좋다. 애초에 들어가서 구동이 안됨


### 방안

자연어(NLP) 해석 신경망을 훔쳐왔다.

**한국어 - 영어 해석은 input과 output shape(글자의 숫자, 주어/동사/목적어)이 달라도 돌아간다.**

그런고로 7일치 6개의 변수 > 2일치 1개의 정답도 대충 되지 않을까 기대함.

안되면 환불손절

In [1]:
from google.colab import drive
!pip install import_ipynb
import import_ipynb
# 구글 드라이브에 자료 전처리 함수, raw_data 올려놓았다.
# 일단 불러오기
drive.mount('/content/drive')

# !cp가 뭔지는 모르지만, 이렇게 해야 함수를 불러올 수 있다고 함.
# lstm_defs.py에 전처리 함수를 저장해 놓았으므로 얘를 MJ로 불러온다.
!cp /content/drive/MyDrive/Jupyter/unlimited_power/working__/LSTM_defs.ipynb .
import LSTM_defs as MJ

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=e70eedfe7b42b96c07e818d8ba7dbed414b6daddabb208a88578be2eb2c03e8e
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
importing Jupyter notebook from LSTM_defs.ipynb


In [2]:
import pandas as pd
from pandas import Series, DataFrame
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
from pandas import DataFrame
from pandas import concat
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [3]:
raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')

In [4]:
# 불러온 데이터를 MJ.py에 미리 정의해 둔 data_loadder 함수에 때려 넣으면
# 자동으로 train_X, train_y, test_X, test_y 4가지가 선언된다.
train_X, train_y, test_X, test_y = MJ.data_loader(raw_data)

In [5]:
train_X.shape

torch.Size([336, 36490, 6])

In [6]:
### parameters

## 학습에 사용할 변수의 개수 5개 + target 변수(7일간)
n_features = 6

In [7]:
class Encoder(nn.Module):

    def __init__(self, input_size, hidden_dim, num_layers=1):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_dim, num_layers=self.num_layers)
        self.hidden = None

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))

    def forward(self, inputs):
        # Push through RNN layer (the ouput is irrelevant)
        _, self.hidden = self.lstm(inputs, self.hidden)
        return self.hidden

In [8]:
class Decoder(nn.Module):

    def __init__(self, hidden_dim, num_layers=1):
        super(Decoder, self).__init__()
        # input_size=1 since the output are single values
        self.lstm = nn.LSTM(1, hidden_dim, num_layers=num_layers)
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, outputs, hidden): #, criterion):
        batch_size, num_steps = outputs.shape
        # Create initial start value/token
        input = torch.tensor([[0.0]] * batch_size, dtype=torch.float)
        # Convert (batch_size, output_size) to (seq_len, batch_size, output_size)
        input = input.unsqueeze(0)

        loss = 0
        for i in range(num_steps):
            # Push current input through LSTM: (seq_len=1, batch_size, input_size=1)
            output, hidden = self.lstm(input, hidden)
            # Push the output of last step through linear layer; returns (batch_size, 1)
            output = self.out(output[-1])
            # Generate input for next step by adding seq_len dimension (see above)
            input = output.unsqueeze(0)
            # Compute loss between predicted value and true value
            loss += tilted_loss(0.5, output, outputs[:, i])
        return loss

In [9]:
# 손실함수
# Quantile 별로 구해야하니까, q 값에 quantile을 넣어야한다.

def tilted_loss(q,y,f):
    e = (y-f)
    return torch.mean(torch.max(q*e, (q-1)*e), axis=-1)

In [ ]:
if __name__ == '__main__':


    # 5 is the number of features of your data points
    encoder = Encoder(n_features, 64)
    decoder = Decoder(64)
    # Create optimizers for encoder and decoder
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
    #criterion = nn.MSELoss()

    
    # Convert (batch_size, seq_len, input_size) to (seq_len, batch_size, input_size)
    inputs = train_X

    # 2 sequences (to match the batch size) of length 6 (for the 6h into the future)
    outputs = train_y

    # Do one complete forward & backward pass
    #
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Reset hidden state of encoder for current batch
    encoder.hidden = encoder.init_hidden(inputs.shape[1])
    # Do forward pass through encoder
    hidden = encoder(inputs)
    # Do forward pass through decoder (decoder gets hidden state from encoder)
    loss = decoder(outputs, hidden)
    # Backpropagation
    loss.backward()
    # Update parameters
    encoder_optimizer.step()
    decoder_optimizer.step()
    print("Loss:", loss.item())